In [9]:
import pandas as pd
import datetime
import numpy as np
import requests
import apimoex
import vectorbt as vbt
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [2]:
money = 100000
start = datetime.datetime(2018, 1, 1)
end = datetime.datetime(2018, 12, 31)
stop_loss = 0.05
fast_window = 30
slow_window = 90

### собираем данные в виде датафрйема

In [3]:
with requests.Session() as session:
    data = apimoex.get_market_candles(session, 'YNDX', interval=60, start=start, end=end)
    df = pd.DataFrame(data)
    df.set_index(pd.to_datetime(df['begin']), inplace=True)
    df.drop(['begin'], axis=1, inplace=True)
    df.drop(columns=['value', 'volume'], inplace=True)
    df = df.rename(columns=lambda x: x.capitalize())

### считаем скользящие быстрые и медленные средние (и их пересечения - сигналы покупки\продажи)

In [4]:
fast_ma = vbt.MA.run(df['Open'], fast_window)
slow_ma = vbt.MA.run(df['Open'], slow_window)

entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)

df['fast'] = fast_ma.ma
df['slow'] = slow_ma.ma
df['entries'] = entries
df['exits'] = exits

### ищем стоплоссы через vectrokastil'

In [5]:
# устанавливаем на открытии позиции стоп лосс
entires_index = df.query('entries == True').index
df.loc[entires_index, 'stop loss'] = df.loc[entires_index, 'Close'].apply(lambda x: x*(1-stop_loss))
# находим, в какие моменты (до стоп лосса) позиция открыта (=1)
portfolio_no_stop_loss = vbt.Portfolio.from_signals(df['Close'], df['entries'], df['exits'], init_cash=money, freq='1h')
df.loc[portfolio_no_stop_loss.assets() > 0,'phase'] = 1
df.loc[portfolio_no_stop_loss.assets() == 0,'phase'] = 0
df.loc[portfolio_no_stop_loss.trades.records_readable['Exit Timestamp'], 'phase'] = 1
# заполняем открытые позиции стоп лоссом (соответсвующим открытию позиции)
play_index = df[df['phase'] == 1].index
df.loc[play_index, 'stop loss'] = df['stop loss'].ffill()
df['stop loss exits'] = df['Low'] <= df['stop loss']
stop_loss_signals = df[df['stop loss exits']].index
# удаляем сигналы стоп лосс, которые срабатывают после стоп лосса (в пределах одного цикла открыто-закрыто)
if len(stop_loss_signals) > 0:
    for i in range(len(stop_loss_signals) - 1):
        if df.loc[stop_loss_signals[i]:stop_loss_signals[i+1]]['phase'].nunique()==1:
            df.loc[stop_loss_signals[i+1], 'stop loss exits'] = False
    if df.loc[stop_loss_signals[-2]:stop_loss_signals[-1]]['phase'].nunique()==1:
        df.loc[stop_loss_signals[-1], 'stop loss exits'] = False
# удаляем сигналы продажи (по скользящим), которые срабатывают после стоп лосса     
    melted = df.reset_index().melt(value_vars=['exits', 'stop loss exits'], id_vars='begin')
    melted = melted[melted['value']].sort_values('begin').reset_index(drop=True)
    if melted.iloc[-1]['variable'] == 'stop loss exits':
        index_to_fix1 = melted[:-1].loc[melted[:-1][melted[:-1]['variable'] == 'stop loss exits'].index + 1, 'begin']
        df.loc[index_to_fix1, 'exits'] = False
    index_to_fix = melted.loc[melted[melted['variable'] == 'stop loss exits'].index + 1, 'begin']
    df.loc[index_to_fix, 'exits'] = False
# находим общие продажи (по стоп лоссам и скользящим) + соответствующие цены продажи
    df['all exits'] = df['stop loss exits'] + df['exits']
    df['price'] = np.where(df['stop loss exits'], df['stop loss'], df['Close'])
else:
    df['all exits'] = df['exits']
    df['price'] = df['Close']

### рисуем чтобы красиво

In [11]:
portfolio = vbt.Portfolio.from_signals(df['price'], df['entries'], df['all exits'], init_cash=money, freq='1h')
fig = portfolio.plot()

fig.add_trace(go.Scatter(x=df.index, y=df['fast'], name= str(fast_window) + ' MA', opacity=0.5))
fig.add_trace(go.Scatter(x=df.index, y=df['slow'], name= str(slow_window) + ' MA', opacity=0.5))

fig.add_trace(go.Scatter(x=df.index, y=df['Low'], name='Low', opacity=0.5))
fig.add_trace(go.Scatter(x=df.index, y=df['stop loss'], name='Stop-Loss level', opacity=0.5))

fig.add_trace(go.Scatter(
    x=df[df['stop loss exits'] == True].index,
    y=df[df['stop loss exits'] == True]['stop loss'],
    mode='markers',
    marker=dict(symbol='triangle-down', color='black', size=10),
    name='Stop-Loss'
))

fig.show()

print('Отчёт по прибыли и сделкам:')
print(portfolio.total_profit())
print(portfolio.stats())
display(portfolio.trades.records_readable)

Отчёт по прибыли и сделкам:
-23776.781668931057
Start                         2018-01-03 09:00:00
End                           2018-12-29 18:00:00
Period                          105 days 06:00:00
Start Value                              100000.0
End Value                            76223.218331
Total Return [%]                       -23.776782
Benchmark Return [%]                     0.573215
Max Gross Exposure [%]                      100.0
Total Fees Paid                               0.0
Max Drawdown [%]                        33.468919
Max Drawdown Duration            90 days 18:00:00
Total Trades                                   16
Total Closed Trades                            15
Total Open Trades                               1
Open Trade PnL                          789.87791
Win Rate [%]                                 20.0
Best Trade [%]                             6.3593
Worst Trade [%]                              -5.0
Avg Winning Trade [%]                    4.046024
Av

,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,0,45.423575,2018-02-14 17:00:00,2201.5,0.0,2018-02-28 17:00:00,2341.50,0.0,6359.300477,0.063593,Long,Closed,0
1,1,0,42.990825,2018-03-09 09:00:00,2474.0,0.0,2018-03-20 14:00:00,2437.50,0.0,-1569.165104,-0.014753,Long,Closed,1
2,2,0,42.511211,2018-03-22 10:00:00,2465.0,0.0,2018-03-26 09:00:00,2377.00,0.0,-3740.986577,-0.035700,Long,Closed,2
3,3,0,47.574929,2018-04-25 14:00:00,2124.0,0.0,2018-04-25 17:00:00,2017.80,0.0,-5052.457440,-0.050000,Long,Closed,3
4,4,0,44.095862,2018-05-03 13:00:00,2177.0,0.0,2018-05-17 09:00:00,2123.00,0.0,-2381.176543,-0.024805,Long,Closed,4
5,5,0,43.684328,2018-06-04 14:00:00,2143.0,0.0,2018-06-11 17:00:00,2196.00,0.0,2315.269382,0.024732,Long,Closed,5
6,6,0,42.139593,2018-06-13 11:00:00,2276.5,0.0,2018-06-25 13:00:00,2220.00,0.0,-2380.887023,-0.024819,Long,Closed,6
7,7,0,40.958799,2018-07-03 13:00:00,2284.0,0.0,2018-07-24 13:00:00,2359.50,0.0,3092.389333,0.033056,Long,Closed,7
8,8,0,39.317448,2018-07-26 13:00:00,2458.0,0.0,2018-07-30 10:00:00,2335.10,0.0,-4832.114325,-0.050000,Long,Closed,8
9,9,0,42.942082,2018-08-22 16:00:00,2138.0,0.0,2018-09-06 18:00:00,2105.50,0.0,-1395.617678,-0.015201,Long,Closed,9


In [7]:
# # проверка: нет ли подряд продаж\покупок?
# melted1=df.reset_index().melt(value_vars=['all exits', 'entries'], id_vars=['begin', 'stop loss exits'])
# melted1[melted1['value']].sort_values('begin')

In [8]:
df.loc[portfolio.assets() > 0,'status'] = 1
df.loc[portfolio.assets() == 0,'status'] = 0
position_df = pd.DataFrame({'In Position':df['status']}, index=df.index)
fig = position_df.vbt.plot()
fig.show()